In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import sys

# add root folder to path
folder = "../../"
sys.path.append(folder)
from src.utils import load_data
from src.utils import evaluate_model

users,movies,ratings = load_data('../../data/movies.csv')

# Split data into train and test by timestamp
ratings['Timestamp'] = pd.to_datetime(ratings['Timestamp'], unit='s')
split_point = int(len(ratings) * 0.8)
sorted_ratings = ratings.sort_values(by='Timestamp')
train_ratings = sorted_ratings[:split_point]
test_ratings = sorted_ratings[split_point:]

item_user_matrix = train_ratings.pivot(index='MovieID', columns='UserID', values='Rating')
item_user_matrix = item_user_matrix.fillna(0)

item_similarity = cosine_similarity(item_user_matrix)
item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

def get_item_recommendations(user_id, item_user_matrix, item_similarity_df, n_recommendations=10):
    if user_id not in item_user_matrix.columns:
        return []

    user_ratings = item_user_matrix.loc[:, user_id]
    weighted_ratings = item_similarity_df.dot(user_ratings)
    sum_of_similarities = item_similarity_df.dot(user_ratings > 0).replace(0, 1)  # Prevent division by zero
    weighted_ratings = weighted_ratings / sum_of_similarities
    already_rated = user_ratings > 0
    weighted_ratings = weighted_ratings[~already_rated]
    recommendations = weighted_ratings.sort_values(ascending=False).head(n_recommendations)
    recommended_movie_ids = recommendations.index

    return recommended_movie_ids

user_id = test_ratings['UserID'].iloc[0]
recommended_movies = get_item_recommendations(user_id, item_user_matrix, item_similarity_df)
print(movies[movies['MovieID'].isin(recommended_movies)][['Title', 'Genres']])

precision, recall, f1 = evaluate_model(test_ratings, item_user_matrix, item_similarity_df, get_item_recommendations)
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1 Score: {f1:.2f}')


